In [30]:
# (_.~" IMPORTS "~._) 

# Import des librairies
import numpy as np
import pandas as pd
import datetime

# Suppression des warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None




# (_.~" IMPORT SCRAPPING NINTENDO "~._) 

# Import pour df
dfNinSoft = pd.read_csv("nintendo_bestsellers_scrapping.csv")
dfNinHard = pd.read_csv("nintendo_hardware_sales.csv")

# Création df avec seulement les jeux de la Switch
df = dfNinSoft[dfNinSoft["Hardware"]=="switch"]
# Suppression de la colonne Hardware puisque une seule console
df = df.drop(["Hardware"], axis=1)
# Séléction des données sur la Switch depuis dfNinHard
dfNinSwitch = dfNinHard[['Date','Hardware','Units']][dfNinHard['Hardware']=='Nintendo Switch'].copy()
dfNinTotal = dfNinHard[['Date','Hardware','Softwares']][dfNinHard['Hardware']=='Nintendo Switch'].copy()
# Réorganisation colonness
dfNinSwitch.rename(columns = { "Hardware": "Quality","Units": "Sales"},inplace = True)
dfNinTotal.rename(columns = {"Hardware": "Quality","Softwares": "Sales"},inplace = True)
df.rename(columns = {"Game": "Quality"},inplace = True)
# Changements des valeurs pour la comparaison
df.replace('Pokémon Sword/ Pokémon Shield', 'Pokémon Sword/Pokémon Shield', inplace = True)
dfNinSwitch.replace('Nintendo Switch', 'Switch', inplace=True)
dfNinTotal.replace('Nintendo Switch', 'Total', inplace=True)

# Assemblage des données dans df
df = pd.concat([df, dfNinSwitch, dfNinTotal])

# Formatage de la colonnes Date en datetime
df["Date"] = pd.to_datetime(df["Date"], format='%Y-%m-%d')
# Création des colonnes "Year" et "Month"
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.strftime('%m').astype('int')
# Instance de Diff
df["Diff"] = df["Sales"]
# Différence entre chaque vente par "Quality"
for i in df["Quality"].unique():
    df["Diff"][df["Quality"]==i] = df["Sales"][df["Quality"]==i].diff()
# Remplissage des valeurs NaN par Sales (égale à la première apparition en Vente)
df["Diff"].fillna(df["Sales"],inplace=True)





# (_.~" IMPORT DES DIFFERENTS DATAFRAME "~._) 

df_1 = df
df_2 = pd.read_csv("AMAZON ET METACRITIC SA NINTENDO TOP5 pour Julien.csv")
df_3 = pd.read_csv("top5_nintendo_games_groupby_Month.csv")



# --- FORMATAGE df_1 ---

# Ajout d'une colonne Game selon la Quality
df_1.loc[df_1['Quality'] == "The Legend of Zelda: Breath of the Wild", 'Game'] = 'BOTW' 
df_1.loc[df_1['Quality'] == "Animal Crossing: New Horizons", 'Game'] = 'ACNH' 
df_1.loc[df_1['Quality'] == "Super Smash Bros. Ultimate", 'Game'] = 'SSBU' 
df_1.loc[df_1['Quality'] == "Mario Kart 8 Deluxe", 'Game'] = 'MKD8' 
df_1.loc[df_1['Quality'] == "Pokémon Brilliant Diamond / Pokémon Shining Pearl", 'Game'] = 'POKE'
df_1.loc[df_1['Quality'] == "Pokémon Scarlet/ Pokémon Violet", 'Game'] = 'POKE'
df_1.loc[df_1['Quality'] == "Pokémon Sword/Pokémon Shield", 'Game'] = 'POKE'
df_1.loc[df_1['Quality'] == "Pokémon: Let's Go, Pikachu!/ Pokémon: Let's Go, Eevee!", 'Game'] = 'POKE'
df_1.loc[df_1['Quality'] == "Total", 'Game'] = 'TOTL'
df_1.loc[df_1['Quality'] == "Switch", 'Game'] = 'SWCH'
# Suppression des colonnes inutiles
df_1 = df_1.drop(["Date","Quality"], axis=1)
# Suppression des jeux non étudiés
df_1 = df_1.dropna(subset=['Game'])
# Réorganisation des colonnes
df_1 = df_1[['Year', 'Month', 'Game', 'Sales', 'Diff']].sort_values(by=['Year','Month'], ignore_index=True)
#Affichage
display(df_1.head(8))



# --- FORMATAGE df_2 ---

# Renommage des colonnes en vue du concat
df_2 = df_2.rename(columns={"month": "Month", 
                            "year": "Year",
                            "title":"Game",
                            "score mean":"mean",
                            'reviews count':"count"})
# Création des nouvelles colonnes à partir de la colonne "sources"
df_2amz = df_2.loc[df_2['sources'] == "AMAZON"]
df_2amz = df_2amz.drop(["sources"], axis=1)
df_2mtc = df_2.loc[df_2['sources'] == "METACRITIC"]
df_2mtc = df_2mtc.drop(["sources"], axis=1)
# Ajout des données crées
df_2 = pd.merge(df_2amz, df_2mtc,  how='left', 
                left_on=['Game','Year','Month'], right_on = ['Game','Year','Month'],
                suffixes=('AMZ', 'MTC'))
#Affichage
display(df_2[df_2['Year']>=2023].head(20))



# --- FORMATAGE df_3 ---

df_3["Date"] = pd.to_datetime(df_3["Date"], format='%Y-%m-%d')
# Création des colonnes "Year" et "Month"
df_3["Year"] = df_3["Date"].dt.year
df_3["Month"] = df_3["Date"].dt.strftime('%m').astype('int')
# Renommage des colonnes en vue du concat
df_3 = df_3.rename(columns={"Game Name":"Game"})
#Suppression des colonnes inutiles
df_3 = df_3.drop(["Date","Reply_SUM.1","Like_SUM.1","Retweet_SUM.1",
                  "Sentiment_VADER_Label_<LAMBDA>","Sentiment_NLTK_Label_<LAMBDA>"], axis=1)
#Affichage
display(df_3.head(8))




# (_.~" CONCATENATION DES DATAFRAME "~._) 

df_final = pd.merge(df_3, df_2, how ='outer',
                    left_on=['Game','Year','Month'], 
                    right_on = ['Game','Year','Month'],
                    suffixes=('',''))
df_final = pd.merge(df_final, df_1, how ='outer',
                    left_on=['Game','Year','Month'], 
                    right_on = ['Game','Year','Month'])

# Réorganisation des colonnes
df_final = df_final[['Game', 'Year', 'Month', 'Sales', 'Diff', 
                     'Reply_SUM', 'Reply_MEAN', 'Reply_MAX', 'Reply_MIN', 'Like_SUM',
                     'Like_MEAN', 'Like_MAX', 'Like_MIN', 'Retweet_SUM', 'Retweet_MEAN',
                     'Retweet_MAX', 'Retweet_MIN', 'User Followers count_MEAN',
                     'User Followers count_MAX', 'User Followers count_MIN',
                     'User Followers count_SUM', 'User Following count_MEAN',
                     'User Following count_MAX', 'User Following count_MIN',
                     'User Following count_SUM', 'Sentiment_TEXTBLOB_MEAN',
                     'Sentiment_VADER_MEAN', 'Sentiment_NLTK_MEAN', 
                     'meanAMZ', 'countAMZ', 'meanMTC', 'countMTC']
                     ].sort_values(by=['Game','Year','Month'], ignore_index=True)
#Affichage
print(df_final.columns)
display(df_final.head(20))



# (_.~" EXPORT "~._)

print(df_final.isna().sum())

csvName = 'finalDataset.csv'
df_final.to_csv(csvName, index=False)

,Year,Month,Game,Sales,Diff
0,2017,3,BOTW,2.76,2.76
1,2017,3,SWCH,2.74,2.74
2,2017,3,TOTL,5.46,5.46
3,2017,6,BOTW,3.92,1.16
4,2017,6,MKD8,3.54,3.54
5,2017,6,SWCH,4.70,1.96
6,2017,6,TOTL,13.60,8.14
7,2017,9,BOTW,4.70,0.78


,Game,Year,Month,meanAMZ,countAMZ,meanMTC,countMTC
36,ACNH,2023,1,8.0,28,7.0,6.0
37,ACNH,2023,2,7.0,9,8.0,5.0
38,ACNH,2023,3,8.0,1,NaN,NaN
39,ACNH,2023,5,10.0,1,0.0,1.0
40,ACNH,2023,6,9.0,3,10.0,2.0
41,ACNH,2023,7,10.0,3,4.0,1.0
42,ACNH,2023,8,9.0,7,NaN,NaN
43,ACNH,2023,9,6.0,2,NaN,NaN
44,ACNH,2023,11,10.0,2,7.0,1.0
45,ACNH,2023,12,10.0,1,NaN,NaN


,Game,Reply_SUM,Reply_MEAN,Reply_MAX,Reply_MIN,Like_SUM,Like_MEAN,Like_MAX,Like_MIN,Retweet_SUM,...,User Followers count_SUM,User Following count_MEAN,User Following count_MAX,User Following count_MIN,User Following count_SUM,Sentiment_TEXTBLOB_MEAN,Sentiment_VADER_MEAN,Sentiment_NLTK_MEAN,Year,Month
0,ACNH,3134,19.710692,222,0,193304,1215.748428,25999,57,51867,...,85593583,1345.886792,106120,0,213996,0.786164,0.139094,0.137810,2019,6
1,ACNH,118,13.111111,29,4,2586,287.333333,1341,55,456,...,574010,2569.555556,15372,55,23126,0.777778,0.416189,0.367333,2019,7
2,ACNH,73,8.111111,29,2,2652,294.666667,848,86,424,...,1271273,1400.777778,7729,5,12607,0.111111,0.212211,0.212211,2019,8
3,ACNH,571,10.773585,115,0,46792,882.867925,14537,67,14525,...,11078670,469.301887,2198,0,24873,0.849057,0.211902,0.211902,2019,9
4,ACNH,281,8.028571,30,0,7893,225.514286,953,70,1739,...,12660970,687.714286,2079,0,24070,0.828571,0.209926,0.209926,2019,10
5,ACNH,174,6.214286,36,0,6576,234.857143,1840,67,1599,...,2328287,472.214286,1523,0,13222,0.750000,0.203286,0.201489,2019,11
6,ACNH,2602,23.654545,928,0,198344,1803.127273,68851,60,52843,...,38445313,735.690909,9970,0,80926,0.818182,0.223177,0.223177,2019,12
7,ACNH,18373,46.279597,4376,0,407439,1026.294710,77913,44,198349,...,91771647,578.219144,15372,0,229553,0.871537,0.244784,0.244784,2020,1


Index(['Game', 'Year', 'Month', 'Sales', 'Diff', 'Reply_SUM', 'Reply_MEAN',
       'Reply_MAX', 'Reply_MIN', 'Like_SUM', 'Like_MEAN', 'Like_MAX',
       'Like_MIN', 'Retweet_SUM', 'Retweet_MEAN', 'Retweet_MAX', 'Retweet_MIN',
       'User Followers count_MEAN', 'User Followers count_MAX',
       'User Followers count_MIN', 'User Followers count_SUM',
       'User Following count_MEAN', 'User Following count_MAX',
       'User Following count_MIN', 'User Following count_SUM',
       'Sentiment_TEXTBLOB_MEAN', 'Sentiment_VADER_MEAN',
       'Sentiment_NLTK_MEAN', 'meanAMZ', 'countAMZ', 'meanMTC', 'countMTC'],
      dtype='object')


,Game,Year,Month,Sales,Diff,Reply_SUM,Reply_MEAN,Reply_MAX,Reply_MIN,Like_SUM,...,User Following count_MAX,User Following count_MIN,User Following count_SUM,Sentiment_TEXTBLOB_MEAN,Sentiment_VADER_MEAN,Sentiment_NLTK_MEAN,meanAMZ,countAMZ,meanMTC,countMTC
0,ACNH,2019,6,NaN,NaN,3134.0,19.710692,222.0,0.0,193304.0,...,106120.0,0.0,213996.0,0.786164,0.139094,0.137810,NaN,NaN,NaN,NaN
1,ACNH,2019,7,NaN,NaN,118.0,13.111111,29.0,4.0,2586.0,...,15372.0,55.0,23126.0,0.777778,0.416189,0.367333,NaN,NaN,NaN,NaN
2,ACNH,2019,8,NaN,NaN,73.0,8.111111,29.0,2.0,2652.0,...,7729.0,5.0,12607.0,0.111111,0.212211,0.212211,NaN,NaN,NaN,NaN
3,ACNH,2019,9,NaN,NaN,571.0,10.773585,115.0,0.0,46792.0,...,2198.0,0.0,24873.0,0.849057,0.211902,0.211902,NaN,NaN,NaN,NaN
4,ACNH,2019,10,NaN,NaN,281.0,8.028571,30.0,0.0,7893.0,...,2079.0,0.0,24070.0,0.828571,0.209926,0.209926,NaN,NaN,NaN,NaN
5,ACNH,2019,11,NaN,NaN,174.0,6.214286,36.0,0.0,6576.0,...,1523.0,0.0,13222.0,0.750000,0.203286,0.201489,NaN,NaN,NaN,NaN
6,ACNH,2019,12,NaN,NaN,2602.0,23.654545,928.0,0.0,198344.0,...,9970.0,0.0,80926.0,0.818182,0.223177,0.223177,NaN,NaN,NaN,NaN
7,ACNH,2020,1,NaN,NaN,18373.0,46.279597,4376.0,0.0,407439.0,...,15372.0,0.0,229553.0,0.871537,0.244784,0.244784,9.0,26.0,3.0,99.0
8,ACNH,2020,2,NaN,NaN,44410.0,48.324266,5728.0,0.0,992681.0,...,15372.0,0.0,531132.0,0.838955,0.260102,0.257575,9.0,36.0,4.0,71.0
9,ACNH,2020,3,11.77,11.77,143369.0,121.602205,22655.0,0.0,1500013.0,...,106120.0,0.0,1211545.0,0.767600,0.267048,0.266127,10.0,164.0,4.0,2026.0


Game                           0
Year                           0
Month                          0
Sales                        343
Diff                         343
Reply_SUM                    142
Reply_MEAN                   142
Reply_MAX                    142
Reply_MIN                    142
Like_SUM                     142
Like_MEAN                    142
Like_MAX                     142
Like_MIN                     142
Retweet_SUM                  142
Retweet_MEAN                 142
Retweet_MAX                  142
Retweet_MIN                  142
User Followers count_MEAN    142
User Followers count_MAX     142
User Followers count_MIN     142
User Followers count_SUM     142
User Following count_MEAN    142
User Following count_MAX     142
User Following count_MIN     142
User Following count_SUM     142
Sentiment_TEXTBLOB_MEAN      142
Sentiment_VADER_MEAN         142
Sentiment_NLTK_MEAN          142
meanAMZ                      163
countAMZ                     163
meanMTC   